In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn

import gym
from gym import wrappers

import torch

import tianshou as ts

import TeachMyAgent as tma

import copy


In [2]:
import argparse
import os
import pprint

import gym
import numpy as np
import torch

from torch.utils.tensorboard import SummaryWriter


from tianshou.data import Collector, VectorReplayBuffer
from tianshou.env import SubprocVectorEnv
from tianshou.policy import SACPolicy
from tianshou.trainer import offpolicy_trainer
from tianshou.utils import TensorboardLogger
from tianshou.utils.net.common import Net
from tianshou.utils.net.continuous import ActorProb, Critic

In [3]:
import numpy as np
import time
import gym
import TeachMyAgent.environments

env = gym.make('parametric-continuous-parkour-v0', agent_body_type='fish', movable_creepers=True)

/Users/akarshkumar0101/miniconda3/envs/qd-acl/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/akarshkumar0101/miniconda3/envs/qd-acl/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/akarshkumar0101/miniconda3/envs/qd-acl/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/akarsh

INFO:tensorflow:Restoring parameters from /Users/akarshkumar0101/TeachMyAgent/TeachMyAgent/environments/envs/PCGAgents/CPPN/weights/same_ground_ceiling_cppn/


/Users/akarshkumar0101/miniconda3/envs/qd-acl/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
env.set_environment(input_vector=np.zeros(3), water_level = 0.1)
env.reset()

while True:
    _, _, d, _ = env.step(env.action_space.sample())
    # env.render()
    # time.sleep(0.1)
    


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/akarshkumar0101/miniconda3/envs/qd-acl/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-17540322d381>", line 7, in <module>
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/akarshkumar0101/miniconda3/envs/qd-acl/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/akarshkumar0101/miniconda3/envs/qd-acl/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context,

TypeError: object of type 'NoneType' has no len()

In [4]:


def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--task', type=str, default="BipedalWalkerHardcore-v3")
    parser.add_argument('--seed', type=int, default=0)
    parser.add_argument('--buffer-size', type=int, default=1000000)
    parser.add_argument('--actor-lr', type=float, default=3e-4)
    parser.add_argument('--critic-lr', type=float, default=1e-3)
    parser.add_argument('--gamma', type=float, default=0.99)
    parser.add_argument('--tau', type=float, default=0.005)
    parser.add_argument('--alpha', type=float, default=0.1)
    parser.add_argument('--auto-alpha', type=int, default=1)
    parser.add_argument('--alpha-lr', type=float, default=3e-4)
    parser.add_argument('--epoch', type=int, default=100)
    parser.add_argument('--step-per-epoch', type=int, default=100000)
    parser.add_argument('--step-per-collect', type=int, default=10)
    parser.add_argument('--update-per-step', type=float, default=0.1)
    parser.add_argument('--batch-size', type=int, default=128)
    parser.add_argument('--hidden-sizes', type=int, nargs='*', default=[128, 128])
    parser.add_argument('--training-num', type=int, default=10)
    parser.add_argument('--test-num', type=int, default=100)
    parser.add_argument('--logdir', type=str, default='log')
    parser.add_argument('--render', type=float, default=0.)
    parser.add_argument('--n-step', type=int, default=4)
    parser.add_argument(
        '--device', type=str, default='cuda' if torch.cuda.is_available() else 'cpu'
    )
    parser.add_argument('--resume-path', type=str, default=None)
    return parser.parse_args("")



In [5]:
class Wrapper(gym.Wrapper):
    """Env wrapper for reward scale, action repeat and removing done penalty"""

    def __init__(self, env, action_repeat=3, reward_scale=5, rm_done=True):
        super().__init__(env)
        self.action_repeat = action_repeat
        self.reward_scale = reward_scale
        self.rm_done = rm_done

    def step(self, action):
        r = 0.0
        for _ in range(self.action_repeat):
            obs, reward, done, info = self.env.step(action)
            # remove done reward penalty
            if not done or not self.rm_done:
                r = r + reward
            if done:
                break
        # scale reward
        return obs, self.reward_scale * r, done, info

In [6]:
args = get_args()

env = Wrapper(gym.make(args.task))

args.state_shape = env.observation_space.shape or env.observation_space.n
args.action_shape = env.action_space.shape or env.action_space.n
args.max_action = env.action_space.high[0]

In [7]:
train_envs = SubprocVectorEnv(
    [lambda: Wrapper(gym.make(args.task)) for _ in range(args.training_num)]
)
# test_envs = gym.make(args.task)
test_envs = SubprocVectorEnv(
    [
        lambda: Wrapper(gym.make(args.task), reward_scale=1, rm_done=False)
        for _ in range(args.test_num)
    ]
)

# seed
np.random.seed(args.seed)
torch.manual_seed(args.seed)
train_envs.seed(args.seed)
test_envs.seed(args.seed);

In [8]:
# model
net_a = Net(args.state_shape, hidden_sizes=args.hidden_sizes, device=args.device)
actor = ActorProb(
    net_a,
    args.action_shape,
    max_action=args.max_action,
    device=args.device,
    unbounded=True
).to(args.device)
actor_optim = torch.optim.Adam(actor.parameters(), lr=args.actor_lr)

net_c1 = Net(
    args.state_shape,
    args.action_shape,
    hidden_sizes=args.hidden_sizes,
    concat=True,
    device=args.device
)
critic1 = Critic(net_c1, device=args.device).to(args.device)
critic1_optim = torch.optim.Adam(critic1.parameters(), lr=args.critic_lr)

net_c2 = Net(
    args.state_shape,
    args.action_shape,
    hidden_sizes=args.hidden_sizes,
    concat=True,
    device=args.device
)
critic2 = Critic(net_c2, device=args.device).to(args.device)
critic2_optim = torch.optim.Adam(critic2.parameters(), lr=args.critic_lr)

if args.auto_alpha:
    target_entropy = -np.prod(env.action_space.shape)
    log_alpha = torch.zeros(1, requires_grad=True, device=args.device)
    alpha_optim = torch.optim.Adam([log_alpha], lr=args.alpha_lr)
    args.alpha = (target_entropy, log_alpha, alpha_optim)

policy = SACPolicy(
    actor,
    actor_optim,
    critic1,
    critic1_optim,
    critic2,
    critic2_optim,
    tau=args.tau,
    gamma=args.gamma,
    alpha=args.alpha,
    estimation_step=args.n_step,
    action_space=env.action_space
)


In [13]:
# load a previous policy
if args.resume_path:
    policy.load_state_dict(torch.load(args.resume_path))
    print("Loaded agent from: ", args.resume_path)

# collector
train_collector = Collector(
    policy,
    train_envs,
    VectorReplayBuffer(args.buffer_size, len(train_envs)),
    exploration_noise=True
)
test_collector = Collector(policy, test_envs)
# train_collector.collect(n_step=args.buffer_size)
# log
log_path = os.path.join(args.logdir, args.task, 'sac')
# writer = SummaryWriter(log_path)
# logger = TensorboardLogger(writer)

def save_fn(policy):
    print('saving')
    # torch.save(policy.state_dict(), os.path.join(log_path, 'policy.pth'))

def stop_fn(mean_rewards):
    return mean_rewards >= env.spec.reward_threshold

In [14]:
# trainer
result = offpolicy_trainer(
    policy,
    train_collector,
    test_collector,
    args.epoch,
    args.step_per_epoch,
    args.step_per_collect,
    args.test_num,
    args.batch_size,
    update_per_step=args.update_per_step,
    test_in_train=False,
    stop_fn=stop_fn,
    save_fn=save_fn,
    # logger=logger
)


saving


Epoch #1: 100001it [08:11, 203.33it/s, alpha=0.104, env_step=100000, len=0, loss/actor=-40.498, loss/alpha=-0.782, loss/critic1=176.738, loss/critic2=174.742, n/ep=0, n/st=10, rew=0.00]                               


Epoch #1: test_reward: -47.791163 ± 29.421405, best_reward: -21.486006 ± 35.335554 in #0


Epoch #2: 100001it [07:46, 214.48it/s, alpha=0.199, env_step=200000, len=0, loss/actor=-62.109, loss/alpha=0.093, loss/critic1=266.739, loss/critic2=260.335, n/ep=0, n/st=10, rew=0.00]                                 


Epoch #2: test_reward: -59.329818 ± 34.202553, best_reward: -21.486006 ± 35.335554 in #0


...

Epoch #8: 100001it [07:41, 216.63it/s, alpha=0.298, env_step=800000, len=0, loss/actor=-155.390, loss/alpha=-0.189, loss/critic1=678.853, loss/critic2=701.587, n/ep=0, n/st=10, rew=0.00]                                


Epoch #8: test_reward: 57.743687 ± 125.716524, best_reward: 66.028082 ± 125.914366 in #6


Epoch #9: 100001it [06:44, 247.49it/s, alpha=0.294, env_step=900000, len=0, loss/actor=-162.459, loss/alpha=0.035, loss/critic1=693.644, loss/critic2=716.336, n/ep=0, n/st=10, rew=0.00]                                 


saving
Epoch #9: test_reward: 72.107249 ± 134.198899, best_reward: 72.107249 ± 134.198899 in #9


Epoch #10:  12%|#2        | 12050/100000 [00:50<06:06, 239.87it/s, alpha=0.312, env_step=912040, len=0, loss/actor=-165.178, loss/alpha=-0.082, loss/critic1=713.259, loss/critic2=727.375, n/ep=0, n/st=10, rew=0.00]     

KeyboardInterrupt



In [ ]:
pprint.pprint(result)
# Let's watch its performance!
policy.eval()
test_envs.seed(args.seed)
test_collector.reset()
result = test_collector.collect(n_episode=args.test_num, render=args.render)
rews, lens = result["rews"], result["lens"]
print(f"Final reward: {rews.mean()}, length: {lens.mean()}")

In [10]:
import matplotlib.pyplot as plt
import gym

env = gym.make('CartPole-v1')

env.gravity = 20
print(env.gravity)
env.print_gravity()

env.set_gravity(40)
print(env.gravity)
env.print_gravity()


20
9.8
20
40


In [4]:
import matplotlib.pyplot as plt
import gym
import numpy as np


for gravity in np.linspace(1, 50, 20):
    env = gym.make('CartPole-v1')
 
    np.random.seed(0)
    env.seed(0)

    obs = env.reset()

    done = False

    eprew = 0
    eplen = 0
    while not done:
        env.set_gravity(gravity)
        
        obs, rew, done, info = env.step(eplen%2)
        eprew += rew
        eplen += 1

    env.close()
    eplen, eprew
    
    print(eprew)
    print(env.state)


56.0
(-0.10434375041651943, 0.03906063338627813, 0.21477228615085472, 0.1514375271789289)
41.0
(-0.08893778710637525, -0.16590066610000895, 0.2140352901972855, 0.6700038126959267)
35.0
(-0.08357150201210116, -0.17310259752823062, 0.21538388606672643, 0.8313309310765957)
31.0
(-0.07971880968863711, -0.1780293823019942, 0.21013077059874896, 0.9404867148296607)
29.0
(-0.07830760800256713, -0.18442663041693377, 0.2190678708087497, 1.0857062251349299)
27.0
(-0.07643739300199816, -0.18865499807711522, 0.21770639374702178, 1.180067122762774)
26.0
(-0.07799230889326551, -0.0006236761621742082, 0.23210123800057633, 1.0438558939224978)
24.0
(-0.07543778581263856, -0.000660368333622291, 0.21537995433144305, 1.0406792730673975)
23.0
(-0.072625177660847, -0.19896599491844436, 0.21338378626713403, 1.409641667687952)
22.0
(-0.07354612031091992, -0.006840778303437101, 0.2135435471586821, 1.178639142477889)
22.0
(-0.07449019971258565, -0.01576179390213972, 0.23471173567526246, 1.384765219844419)
21.0
(

In [2]:
class Net(nn.Module):
    def __init__(self, state_shape, action_shape):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(np.prod(state_shape), 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, np.prod(action_shape)),
        )

    def forward(self, obs, state=None, info={}):
        if not isinstance(obs, torch.Tensor):
            obs = torch.tensor(obs, dtype=torch.float)
        batch = obs.shape[0]
        logits = self.model(obs.view(batch, -1))
        return logits, state

In [18]:
env = gym.make('CartPole-v0')
env.print_gravity()
env.env.gravity = 20
env.print_gravity()

9.8
20


In [3]:
env = gym.make('CartPole-v0')
# train_envs = ts.env.DummyVectorEnv([lambda: gym.make('CartPole-v0') for _ in range(10)])
# test_envs = ts.env.DummyVectorEnv([lambda: gym.make('CartPole-v0') for _ in range(100)])
train_envs = gym.make('CartPole-v0')
test_envs = gym.make('CartPole-v0')
state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n

In [4]:

def random_policy():
    net = Net(state_shape, action_shape)
    optim = torch.optim.Adam(net.parameters(), lr=1e-3)
    policy = ts.policy.DQNPolicy(net, optim, discount_factor=0.9, estimation_step=3, target_update_freq=320)
    return policy


In [40]:
# pre-collect at least 5000 transitions with random action before training
train_collector.collect(n_step=5000, random=True)

policy.set_eps(0.1)
for i in range(int(1e6)):  # total step
    collect_result = train_collector.collect(n_step=10)

    # once if the collected episodes' mean returns reach the threshold,
    # or every 1000 steps, we test it on test_collector
    if collect_result['rews'].mean() >= env.spec.reward_threshold or i % 1000 == 0:
        policy.set_eps(0.05)
        result = test_collector.collect(n_episode=100)
        print('testing collector rewards: ', result['rews'].mean())
        if result['rews'].mean() >= env.spec.reward_threshold:
            print(f'Finished training! Test mean returns: {result["rews"].mean()}')
            break
        else:
            # back to training eps
            policy.set_eps(0.1)

    # train policy with a sampled batch data from buffer
    losses = policy.update(64, train_collector.buffer)

testing collector rewards:  9.45


/Users/akarshkumar0101/miniconda3/envs/tmacenv/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice.
  # Remove the CWD from sys.path while we load stuff.


testing collector rewards:  199.99
Finished training! Test mean returns: 199.99


In [5]:
espace = np.linspace(0, 50, 10)[:, None]

In [6]:

def train_policy(policy, task):
    policy.set_eps(0.1)
    train_envs.set_gravity(task[0])
    
    train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(20000, 10), exploration_noise=True)

    # pre-collect at least 5000 transitions with random action before training
    # print('collecting')
    train_collector.collect(n_step=100, random=True)
    # print('training')
    for i in range(int(1e2)):  # total step
        collect_result = train_collector.collect(n_step=10)
        # train policy with a sampled batch data from buffer
        losses = policy.update(64, train_collector.buffer)
    # print('done training')
    return policy

def test_policy(policy):
    policy.set_eps(0.05)
    results = []
    for task in espace:
        # test_envs.gravity = e[0]
        test_envs.set_gravity(task[0])
        test_envs.reset()
        test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)
        
        result = test_collector.collect(n_episode=10)
        results.append(result['rews'])
        
    results = np.array(results)
    return results.mean(), results



In [24]:
p = random_policy()

for i in range(100):
    train_policy(p, [9.8])
    print(test_policy(p), test_policy(p), test_policy(p), test_policy(p), test_policy(p))
    

10.09 10.3 10.19 10.06 10.05
10.13 10.15 10.13 10.13 10.07
9.65 9.57 9.42 9.4 9.46
9.53 9.71 9.59 9.49 9.45
49.21 50.09 49.72 48.08 45.88
62.81 62.59 62.68 63.25 62.38
194.87 192.52 190.68 192.73 193.72
165.76 169.88 168.11 165.65 166.13
151.49 152.21 156.86 152.49 151.74
171.33 181.09 176.6 177.36 181.35
185.86 187.4 189.23 185.89 186.83
189.62 189.05 180.85 184.92 189.92
194.45 190.81 197.16 192.45 192.04
195.1 193.05 192.01 190.94 190.92
191.3 186.44 188.4 189.14 187.07
197.42 199.64 196.59 196.08 196.71
185.59 185.55 184.37 185.22 182.4
184.32 185.92 190.48 190.35 187.76
198.31 198.72 198.47 199.2 197.48
195.28 196.23 195.48 194.91 196.24
161.57 161.54 159.34 163.26 163.64
187.56 189.17 187.47 189.59 187.2
158.4 155.48 159.91 157.98 157.45
120.73 115.97 117.07 119.31 121.12
79.75 79.36 79.6 81.37 79.51
96.19 95.89 96.41 94.63 94.25
116.5 117.43 118.46 120.35 117.12
115.17 113.99 113.71 111.14 115.2
119.92 121.8 121.05 120.67 122.35
152.06 151.45 151.03 149.87 152.89
198.1 197.59 19

In [8]:
scores = np.zeros(espace.shape[:-1])
policies = np.empty(espace.shape[:-1], dtype=object)
coors_filled = set()

best_score = 0

history = []

for i in range(1000):
    print(i, (i*100)/1000., '%')
    if i<20:
        coor_old = None
        p_old = random_policy()
    else:
        coor_old = np.random.choice(list(coors_filled))#random_filled_coordinate()
        p_old = copy.deepcopy(policies[coor_old])
        
    # TODO: maybe add random mutation noise to escape local minima?
    
    coor_new = np.random.choice(len(espace)) #random_coordinate()
    p_new = train_policy(p_old, espace[coor_new])
    score_new, _ = test_policy(p_new)
    
    should_transfer = policies[coor_new] is None or score_new > scores[coor_new]
    
    if should_transfer:
        if score_new>best_score:
            best_score = score_new
            print('************ new best score = ', best_score, ' ************')
            
        scores[coor_new] = score_new
        policies[coor_new] = p_new
        coors_filled.add(coor_new)
    
    history.append([coor_old, coor_new, should_transfer, scores.copy()])
    print('transfer: ', should_transfer)
    print(scores)

0 0.0 %
************ new best score =  11.23  ************
transfer:  True
[ 0.   11.23  0.    0.    0.    0.    0.    0.    0.    0.  ]
1 0.1 %
transfer:  True
[ 0.   11.23  0.   11.19  0.    0.    0.    0.    0.    0.  ]
2 0.2 %
************ new best score =  14.85  ************
transfer:  True
[ 0.   11.23  0.   14.85  0.    0.    0.    0.    0.    0.  ]
3 0.3 %
transfer:  True
[ 0.   11.23  0.   14.85 10.45  0.    0.    0.    0.    0.  ]
4 0.4 %
transfer:  False
[ 0.   11.23  0.   14.85 10.45  0.    0.    0.    0.    0.  ]
5 0.5 %
transfer:  True
[ 0.   11.23  0.   14.85 10.45  0.    9.42  0.    0.    0.  ]
6 0.6 %
************ new best score =  29.81  ************
transfer:  True
[29.81 11.23  0.   14.85 10.45  0.    9.42  0.    0.    0.  ]
7 0.7 %
transfer:  True
[29.81 11.23 22.37 14.85 10.45  0.    9.42  0.    0.    0.  ]
8 0.8 %
transfer:  True
[29.81 11.23 22.37 14.85 10.45  0.    9.42  9.33  0.    0.  ]
9 0.9 %
transfer:  False
[29.81 11.23 22.37 14.85 10.45  0.    9.42  9.3

KeyboardInterrupt: 

In [11]:
history

[[None,
  1,
  True,
  array([177.62, 179.05, 186.77, 173.93, 179.81, 176.92, 180.72, 184.25,
         175.85, 175.08])],
 [None,
  3,
  True,
  array([177.62, 179.05, 186.77, 173.93, 179.81, 176.92, 180.72, 184.25,
         175.85, 175.08])],
 [None,
  3,
  True,
  array([177.62, 179.05, 186.77, 173.93, 179.81, 176.92, 180.72, 184.25,
         175.85, 175.08])],
 [None,
  4,
  True,
  array([177.62, 179.05, 186.77, 173.93, 179.81, 176.92, 180.72, 184.25,
         175.85, 175.08])],
 [None,
  3,
  False,
  array([177.62, 179.05, 186.77, 173.93, 179.81, 176.92, 180.72, 184.25,
         175.85, 175.08])],
 [None,
  6,
  True,
  array([177.62, 179.05, 186.77, 173.93, 179.81, 176.92, 180.72, 184.25,
         175.85, 175.08])],
 [None,
  0,
  True,
  array([177.62, 179.05, 186.77, 173.93, 179.81, 176.92, 180.72, 184.25,
         175.85, 175.08])],
 [None,
  2,
  True,
  array([177.62, 179.05, 186.77, 173.93, 179.81, 176.92, 180.72, 184.25,
         175.85, 175.08])],
 [None,
  7,
  True,
  

In [12]:
type(env)

gym.wrappers.time_limit.TimeLimit


so we can like not pair the environment and agents...

keep set of evolving environments that try to challenge the agents
keep set of evolving agents that goal switch envs to create curriculum

env behavioral vector is the vector of scores from the test distribution
maintain diversity in this.
make sure no agent strictly out performs another, etc.

essentially we are creating a unpaired agent QD version of POET which tries to solve the ACL problem
to maximize the performance over a training distribution